# Model registry: MLFlow

Example on how to use the MLFlow model during the training of a model

## 1. Install MLFlow

In [1]:
!pip3 install -r requirements.txt
!pip3 freeze | grep mlflow


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
mlflow==2.20.1
mlflow-skinny==2.20.1



## 2. Start the MLFlow server.

To do this, run the following command in a terminal:

mlflow ui --host 0.0.0.0 --port 5001

## 3. Create a new experiment

In [2]:
import mlflow
mlflow.set_tracking_uri("http://localhost:5001")
mlflow.set_experiment("mlflow-model-training-kddcup99")

<Experiment: artifact_location='mlflow-artifacts:/385808207734345689', creation_time=1744898561915, experiment_id='385808207734345689', last_update_time=1744898561915, lifecycle_stage='active', name='mlflow-model-training-kddcup99', tags={}>

## 4. Log a model experiment result

In [3]:
# Lets first create a simple function to train a model
# The function will receive a set of hyperparameters and return the model and the accuracy
import datetime
from mlflow.models import infer_signature

def get_data():
    from sklearn import datasets

    # X, y = datasets.make_classification(n_samples=1000, n_features=4, n_informative=4, n_redundant=0, n_classes=3, n_clusters_per_class=1, class_sep=0.5, random_state=40)
    # use kddcup99 dataset
    X, y = datasets.load_wine(return_X_y=True)
    return X, y

def train_model(hyperparameters):
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.metrics import accuracy_score
    from sklearn.model_selection import train_test_split
    import mlflow.sklearn
    X, y = get_data()
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=40)
    model = RandomForestClassifier(**hyperparameters)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    signature = infer_signature(X, model.predict(X))
    accuracy = accuracy_score(y_pred, y_test)
    return model, signature, accuracy

# Now lets log the model and the accuracy
date = datetime.datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
with mlflow.start_run(run_name=f"mlflow-training-{date}"):
    
    # Now lets train a model with some hyperparameters
    hyperparameters = {"n_estimators": 10, "max_depth": 5}
    model, signature, accuracy = train_model(hyperparameters)
    
    # Hyperparameters are logged as parameters
    for hyperparameter, value in hyperparameters.items():
        mlflow.log_param(hyperparameter, value)
    
    # Accuracy is logged as a metric 
    mlflow.log_metric("accuracy", accuracy)
    
    # Model is logged as an artifact
    mlflow.sklearn.log_model(model, "model", signature=signature)

🏃 View run mlflow-training-2025-04-17-16-13-07 at: http://localhost:5001/#/experiments/385808207734345689/runs/68cdbfc222554641a2ce908a8381edca
🧪 View experiment at: http://localhost:5001/#/experiments/385808207734345689


## 5. Run a hyperparameter search and log the results

In [4]:

import optuna
import mlflow
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score


X, y = get_data()

def optimize_rf(trial):
    
    with mlflow.start_run(run_name=f"optuna-hp-{trial.number}-{datetime.datetime.now().strftime('%Y-%m-%d-%H-%M-%S')}"):
        # Set the hyperparameter values that we want to optimize
        n_estimators = trial.suggest_int('n_estimators', 1, 100)
        max_depth = trial.suggest_int('max_depth', 2, 10)
        min_samples_split = trial.suggest_int('min_samples_split', 2, 20)
        min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 20)
        max_features = trial.suggest_float('max_features', 0.1, 1.0)
        
        
        # Create a random forest classifier using the suggested hyperparameters
        rf = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth,
                                    min_samples_split=min_samples_split,
                                    min_samples_leaf=min_samples_leaf,
                                    max_features=max_features)
        
        # Use cross-validation to evaluate the performance of the classifier
        scores = cross_val_score(rf, X, y, cv=5)
        
        # Log the hyperparameters and cross-validation scores to MLflow
        mlflow.log_param('n_estimators', n_estimators)
        mlflow.log_param('max_depth', max_depth)
        mlflow.log_param('min_samples_split', min_samples_split)
        mlflow.log_param('min_samples_leaf', min_samples_leaf)
        mlflow.log_param('max_features', max_features)
        mlflow.log_metric('mean_cv_score', scores.mean())
    
    # Return the mean of the cross-validation scores as the objective value
    return scores.mean()

# Create an Optuna study
study = optuna.create_study()

# Run the optimization loop
study.optimize(optimize_rf, n_trials=50)

# Get the best hyperparameter values
best_params = study.best_params

/Users/cokestuyck/Documents/GitHub/EDEM_MDA2425/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2025-04-17 16:13:11,278] A new study created in memory with name: no-name-d7431e59-8aae-4222-a119-7e1979e6f807
[I 2025-04-17 16:13:11,422] Trial 0 finished with value: 0.9553968253968254 and parameters: {'n_estimators': 35, 'max_depth': 9, 'min_samples_split': 20, 'min_samples_leaf': 20, 'max_features': 0.2860544141273841}. Best is trial 0 with value: 0.9553968253968254.
[I 2025-04-17 16:13:11,512] Trial 1 finished with value: 0.972063492063492 and parameters: {'n_estimators': 16, 'max_depth': 9, 'min_samples_split': 16, 'min_samples_leaf': 10, 'max_features': 0.22725954032205847}. Best is trial 0 with value: 0.9553968253968254.


🏃 View run optuna-hp-0-2025-04-17-16-13-11 at: http://localhost:5001/#/experiments/385808207734345689/runs/91eca46b5d70472aaf6a7ba7e0d7493c
🧪 View experiment at: http://localhost:5001/#/experiments/385808207734345689
🏃 View run optuna-hp-1-2025-04-17-16-13-11 at: http://localhost:5001/#/experiments/385808207734345689/runs/b249696f31fd49059f7749bc0813ccb5
🧪 View experiment at: http://localhost:5001/#/experiments/385808207734345689


[I 2025-04-17 16:13:11,734] Trial 2 finished with value: 0.9273015873015872 and parameters: {'n_estimators': 65, 'max_depth': 4, 'min_samples_split': 6, 'min_samples_leaf': 15, 'max_features': 0.6965494272140555}. Best is trial 2 with value: 0.9273015873015872.
[I 2025-04-17 16:13:11,923] Trial 3 finished with value: 0.9719047619047618 and parameters: {'n_estimators': 56, 'max_depth': 2, 'min_samples_split': 12, 'min_samples_leaf': 19, 'max_features': 0.39590816673544094}. Best is trial 2 with value: 0.9273015873015872.


🏃 View run optuna-hp-2-2025-04-17-16-13-11 at: http://localhost:5001/#/experiments/385808207734345689/runs/d8a6e32d3bc7485287d30550c975753c
🧪 View experiment at: http://localhost:5001/#/experiments/385808207734345689
🏃 View run optuna-hp-3-2025-04-17-16-13-11 at: http://localhost:5001/#/experiments/385808207734345689/runs/8b2adbef821e4ab98a6c91d2e208eb64
🧪 View experiment at: http://localhost:5001/#/experiments/385808207734345689


[I 2025-04-17 16:13:12,243] Trial 4 finished with value: 0.9220634920634921 and parameters: {'n_estimators': 87, 'max_depth': 5, 'min_samples_split': 8, 'min_samples_leaf': 12, 'max_features': 0.8876134900869159}. Best is trial 4 with value: 0.9220634920634921.


...

🏃 View run optuna-hp-44-2025-04-17-16-13-17 at: http://localhost:5001/#/experiments/385808207734345689/runs/718488a8008c4b209d6ef94da4415778
🧪 View experiment at: http://localhost:5001/#/experiments/385808207734345689
🏃 View run optuna-hp-45-2025-04-17-16-13-18 at: http://localhost:5001/#/experiments/385808207734345689/runs/91c676284d664fffb77ed94ba8db3980
🧪 View experiment at: http://localhost:5001/#/experiments/385808207734345689
🏃 View run optuna-hp-46-2025-04-17-16-13-18 at: http://localhost:5001/#/experiments/385808207734345689/runs/690b17a556574ce5b9b05e51437b9897
🧪 View experiment at: http://localhost:5001/#/experiments/385808207734345689


[I 2025-04-17 16:13:18,435] Trial 47 finished with value: 0.9385714285714286 and parameters: {'n_estimators': 39, 'max_depth': 6, 'min_samples_split': 17, 'min_samples_leaf': 10, 'max_features': 0.7290577758436363}. Best is trial 41 with value: 0.8314285714285713.


🏃 View run optuna-hp-47-2025-04-17-16-13-18 at: http://localhost:5001/#/experiments/385808207734345689/runs/f849624be4864b9fbae38a7168f34edf
🧪 View experiment at: http://localhost:5001/#/experiments/385808207734345689


[I 2025-04-17 16:13:18,719] Trial 48 finished with value: 0.9165079365079365 and parameters: {'n_estimators': 63, 'max_depth': 9, 'min_samples_split': 11, 'min_samples_leaf': 15, 'max_features': 0.9214500687450473}. Best is trial 41 with value: 0.8314285714285713.
[I 2025-04-17 16:13:18,813] Trial 49 finished with value: 0.9385714285714286 and parameters: {'n_estimators': 14, 'max_depth': 7, 'min_samples_split': 15, 'min_samples_leaf': 12, 'max_features': 0.7872174300104212}. Best is trial 41 with value: 0.8314285714285713.


🏃 View run optuna-hp-48-2025-04-17-16-13-18 at: http://localhost:5001/#/experiments/385808207734345689/runs/50b4238e216e4a10871af66fd3cb6b2f
🧪 View experiment at: http://localhost:5001/#/experiments/385808207734345689
🏃 View run optuna-hp-49-2025-04-17-16-13-18 at: http://localhost:5001/#/experiments/385808207734345689/runs/6d759853d09f4457a494fad907fd936c
🧪 View experiment at: http://localhost:5001/#/experiments/385808207734345689


# Deliverable: Screenshot of the MLflow UI with the hyperparameter search results

![MLflow](mlflow-hiperparameters.png "MLflow UI with the hyperparameter search results")

## 6. Register a model version using the best hyperparameters

In [5]:
from mlflow.models.signature import infer_signature

with mlflow.start_run(run_name=f"optuna-hp-final"):
    # Create the final model using the best hyperparameters
    final_model = RandomForestClassifier(**best_params)

    # Train the final model on the entire dataset
    final_model.fit(X, y)

    # Infer the model signature
    signature = infer_signature(X, final_model.predict(X))

    # Log the model to the "Models" section
    mlflow.sklearn.log_model(final_model, "random_forest_model_kddcup99", registered_model_name="random_forest_model_kddcup99", signature=signature)

Successfully registered model 'random_forest_model_kddcup99'.
2025/04/17 16:13:20 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: random_forest_model_kddcup99, version 1


🏃 View run optuna-hp-final at: http://localhost:5001/#/experiments/385808207734345689/runs/21f6423c37d04bd09ec46b3b8c509a5d
🧪 View experiment at: http://localhost:5001/#/experiments/385808207734345689


Created version '1' of model 'random_forest_model_kddcup99'.


## 7. Retrieve a model version and use it for prediction

In [6]:
import mlflow.pyfunc

model_name = "random_forest_model_kddcup99"
model_version = 1

model = mlflow.pyfunc.load_model(
    model_uri=f"models:/{model_name}/{model_version}"
)

model.predict(X[:5])

array([0, 0, 0, 0, 0])